In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import tensorflow as tf
import numpy as np
from ray.experimental.tfutils import TensorFlowVariables

/anaconda3/envs/flow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/anaconda3/envs/flow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1600px;height:750px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [4]:
A = np.array([[1.001, 0], [0, 0.5]])
B = np.array([[1.], [1.]])

# for controllability
cont = np.hstack([B, A@B, A@A@B])
assert np.linalg.matrix_rank(cont)==cont.shape[0]

In [17]:
x0 = np.array([[1.], [1.]])  # 2x1 vector 

A = tf.constant(np.array([[1.001, 0], [0, -0.6]]), dtype=tf.float32, name='A')  # system dynamics
B = tf.constant(np.array([[1.], [1.]]), dtype=tf.float32, name='B')

Q = tf.constant([[1.,0.], [0.,1.]], name='Q')  # weighting
R = tf.constant(1., name='R')

x = tf.constant(x0, dtype=tf.float32, name="state")

T = 10

In [18]:
####### Defining network #######
# input: state x
# output: control u

input_layer = tf.placeholder(tf.float32, (None,2), name='in_layer')
# fc1 = tf.layers.dense(inputs=input_layer, units=1, name='fc1', reuse=tf.AUTO_REUSE)
# u = tf.layers.dense(inputs=fc1, units=1, name='fc_out', reuse=tf.AUTO_REUSE)
u = tf.layers.dense(inputs=input_layer, units=1, name='u_out_layer', reuse=tf.AUTO_REUSE)

### LOSS FUNCTION ### 
loss = tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), 
              tf.matmul(tf.transpose(u), tf.multiply(R, u)), name='loss')

# xs = tf.identity(x, name='xs')
# us = tf.constant(0, name='us')
xs = x
us = u

# cond = lambda i, x, l, xs, us: i < T

# def body(i, x, l, xs, us):
#     next_i = i+1
#     next_x = tf.add(tf.matmul(A, x), tf.multiply(u,B))
#     next_l = tf.add(l,
#                     tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x),
#                            tf.matmul(tf.transpose(u), tf.multiply(R, u))))
#     next_xs = tf.concat(xs, next_x)
#     next_us = tf.concat(us, u)
#     return (next_i, next_x, next_l, next_xs, next_us)

# i, xloss_f, traj_f = tf.while_loop(cond, body, 
#                                    loop_vars=[tf.constant(0), x, loss, xs, us],
#                                    shape_invariants=[tf.TensorShape([1,]), tf.TensorShape([2, 1]), 
#                                                      tf.TensorShape([1,]) , tf.TensorShape([2, None]), 
#                                                      tf.TensorShape([1, None])])
# train = tf.train.GradientDescentOptimizer(0.01).minimize(xloss_f.loss)

for i in range(T):
    # LQR loss 
#     x_term = tf.matmul(tf.matmul(tf.transpose(x), Q), x, name='x_term')
#     u_term = tf.matmul(tf.transpose(u), tf.multiply(R, u), name='u_term')
#     loss = tf.add(loss, tf.add(x_term, u_term), name='loss')  # accumulate loss
    
    # Dynamics: advancing the system dynamics
    Ax = tf.matmul(A, x, name='Ax'+str(i))
    Bu = tf.multiply(u, B, name='Bu'+str(i))  # tf.multiply because u is a scalar
    x = tf.add(Ax, Bu, name='state'+str(i))  # next state vector

    loss = tf.add(loss, tf.add(tf.matmul(tf.matmul(tf.transpose(x), Q), x), tf.matmul(tf.transpose(u), tf.multiply(R, u))), name='loss'+str(i))  # accumulate loss    
    
    u = tf.layers.dense(inputs=tf.transpose(x), units=1, name='u_out_layer', reuse=True)
    
#     fc1 = tf.layers.dense(inputs=tf.transpose(x), units=1, name='fc1', reuse=True)
#     u = tf.layers.dense(inputs=fc1, units=1, name='fc_out', reuse=True)
    
    xs = tf.concat([xs, x], 1)
    us = tf.concat([us, u], 1)
    
opt = tf.train.GradientDescentOptimizer(0.0001)
train = opt.minimize(loss)
grads_and_vars = opt.compute_gradients(loss)

In [ ]:
# show_graph(tf.get_default_graph().as_graph_def())

In [54]:
new_weights = v[-1]['weights'].copy()
new_weights['u_out_layer/bias'] = new_weights.pop('fc1/biases')
new_weights['u_out_layer/kernel'] = new_weights.pop('fc1/weights')
del new_weights['fc_out/biases']
del new_weights['fc_out/weights']
new_weights

{'u_out_layer/bias': array([0.01161656], dtype=float32),
 'u_out_layer/kernel': array([[-1.086187  ],
        [-0.03679204]], dtype=float32)}

In [55]:
variables.get_weights()

{'u_out_layer/bias': array([0.], dtype=float32),
 'u_out_layer/kernel': array([[ 0.09326291],
        [-0.65355664]], dtype=float32)}

In [63]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
variables = TensorFlowVariables(loss, sess)
variables.set_weights(new_weights)
for g, v in grads_and_vars:
    print('gradients')
    print(str(sess.run(g, feed_dict={input_layer : x0.T})) + " - " + v.name)
sess.close()

gradients
[[-18.960514]
 [-47.901035]] - u_out_layer/kernel:0
gradients
[-11.397677] - u_out_layer/bias:0


[[-18.960514], [-47.901035]]

In [61]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
#         sess.run(train)
        sess.run(train, feed_dict={input_layer : x0.T})
        if i % 100 == 0:
#             results = sess.run([xs, us, loss, grads_and_vars], feed_dict={input_layer : x0.T})
            results = sess.run([xs, us, loss], feed_dict={input_layer : x0.T})
            labels  = "xs us loss".split(' ')
            print('training iteration', i)
            for label,result in zip(*(labels,results)) :
                print(label)
                print(result)
                print('')
            print('gradients')
            for g, v in grads_and_vars:
                print(str(sess.run(g, feed_dict={input_layer : x0.T})) + " - " + v.name)
            print('----------------------')

training iteration 0
xs
[[ 1.          1.6705077   1.4057143   0.8651598   0.38684186  0.08079776
  -0.06432917 -0.10282908 -0.08857299 -0.05855154 -0.03181436]
 [ 1.          0.0695076  -0.30816853 -0.35705918 -0.26494765 -0.14746237
  -0.05673031 -0.0043974   0.01699736  0.01991161  0.01484877]]

us
[[ 0.6695076  -0.26646397 -0.5419603  -0.47918317 -0.30643097 -0.14520773
  -0.03843558  0.01435892  0.03011003  0.02679574  0.01721367]]

loss
[[9.630793]]

gradients
[[25.11649 ]
 [16.611288]] - u_out_layer/kernel:0
[16.758821] - u_out_layer/bias:0
----------------------
training iteration 100
xs
[[ 1.          1.4136603   0.8282422   0.20646822 -0.12854722 -0.21178775
  -0.17515226 -0.11803137 -0.08181047 -0.0696028  -0.07087626]
 [ 1.         -0.18733984 -0.4744279  -0.33794546 -0.13245463 -0.00363919
   0.03903079  0.03387757  0.01601239  0.00268205 -0.00281309]]

us
[[ 0.41266018 -0.5868318  -0.6226022  -0.33522192 -0.08311197  0.03684728
   0.05729605  0.03633894  0.01228949 -0.001

## compute optimal loss with true LQR ricatti equation formulation

In [22]:
import controlpy

In [25]:
with tf.Session() as sess:
    A_np, B_np, Q_np, R_np = sess.run([A, B, Q, R])

K, P, eig = controlpy.synthesis.controller_lqr_discrete_time(A_np, B_np, Q_np, R_np)
x_np = x0
u_np = (-K@x_np)
xs_np, us_np = np.array(x_np), np.array(u_np)
loss_np = 0
for i in range(T):
    loss_np += x_np.T@Q_np@x_np + u_np.T*R_np*u_np
    x_np = A_np@x_np + B_np@u_np
    u_np = (-K@x_np)
    xs_np = np.hstack([xs_np, x_np])
    us_np = np.hstack([us_np, u_np])

In [26]:
loss_np

array([[3.71948966]])

## Pulling iter_vars

In [10]:
import pickle

In [47]:
with open('/Users/nishant/ray_results/2018-08-26_22-43-29r4a9tt7u/iter_vars.pkl', 'rb') as file:
    v = pickle.load(file)

In [48]:
v[-1]['weights']

{'fc1/biases': array([0.01161656], dtype=float32),
 'fc1/weights': array([[-1.086187  ],
        [-0.03679204]], dtype=float32),
 'fc_out/biases': array([-0.03084663], dtype=float32),
 'fc_out/weights': array([[0.38942182]], dtype=float32)}